In [1]:
import numpy as np 
import keras
from sklearn.model_selection import train_test_split
from supervisedCar import functions as f
from supervisedCar import architecture
import argparse
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

In [3]:
num_classes = 5
resnet_weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

In [4]:
my_new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


In [5]:
my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:


args = {"batch_size": 200,
        "augmentation": True,
       "epochs": 2,
       "val_split":0.2,
       "early_stop":True,
       "patience":2}

In [7]:



# Model path
model_path = 'Models/'

# Get training data from the Axionable track
X_axio = np.load('Datasets/axionable_data/X_train_axio.npy')
Y_axio = np.load('Datasets/axionable_data/Y_train_axio.npy')
print('Axionable data Loaded. Shape = ', np.shape(X_axio))

# Data augmentation of the dataset / Adjust the proportion of each transformation you want to apply.
if args['augmentation']:
    print('Augmenting data... Wait...')
    # Data augmentation 25% of random brightness.
    X_bright, Y_bright = f.generate_brightness(X_axio, Y_axio, proportion=0.25)
    # Data augmentation 25% of night effect.
    X_night, Y_night = f.generate_night_effect(X_axio, Y_axio, proportion=0.25)
    # Data augmentation 25% of horizontal flipping.
    X_flip, Y_flip = f.generate_horizontal_flip(X_axio, Y_axio, proportion=0.25)
    # Data augmentation 25% of random shadows.
    X_shadow, Y_shadow = f.generate_random_shadows(X_axio, Y_axio, proportion=0.25)
    # Data augmentation 25% of chained tranformations (bright + shadows + flip).
    X_chain, Y_chain = f.generate_chained_transformations(X_axio, Y_axio, proportion=0.25)

    # Concatenating Axionable dataset with the transformations.
    X_axio = np.concatenate((X_axio, X_bright, X_night,
                                X_flip, X_shadow, X_chain))

    Y_axio = np.concatenate((Y_axio, Y_bright, Y_night, 
                                Y_flip, Y_shadow, Y_chain)).astype('float32')

    print('Axionable data after augmentation. Shape = ', np.shape(X_axio))
    
# Get training data from IronCar track
# New track - Double chicane
X_chicane = np.load('Datasets/ironcar_data/new_track/x_chicane.npy')
Y_chicane = np.load('Datasets/ironcar_data/new_track/y_chicane.npy')
print('Ironcar new track chicane Loaded. Shape = ', np.shape(X_chicane))


# Old track - Balanced dataset
X_iron = np.load('Datasets/ironcar_data/old_track/balanced_iron_X.npy')
Y_iron = np.load('Datasets/ironcar_data/old_track/balanced_iron_Y.npy')
print('Ironcar old track data Loaded. Shape = ', np.shape(X_iron))

if args['augmentation']:
    print('Augmenting data... Wait...')
    # Augmentation of the "Double Chicane" Proportion=1. Duplicate the original data (aprox 1500 images)
    X_chicane_aug, Y_chicane_aug = f.generate_brightness(X_chicane, Y_chicane, proportion=1)
    # Irocar Balanced Dataset
    # 25% of random bright transformations 
    X_bright_iron, Y_bright_iron = f.generate_brightness(X_iron, Y_iron, proportion=0.25)
    # 25% of lo gamma transformations (darker images)
    X_gamma_iron, Y_gamma_iron = f.generate_low_gamma(X_iron, Y_iron, proportion=0.25, min_=0.7, max_=0.8)

    # Concatenating IronCar dataset with the transformations.
    X_iron = np.concatenate((X_iron, X_chicane_aug, 
                                   X_bright_iron,X_gamma_iron))

    Y_iron = np.concatenate((Y_iron, Y_chicane_aug, 
                                   Y_bright_iron, Y_gamma_iron)).astype('float32')

    print('Ironcar data after augmentation. Shape = ', np.shape(X_iron))

    # Concatenate both augmented datasets in a single one
X = np.concatenate((X_axio, X_iron))
Y = np.concatenate((Y_axio, Y_iron))
print('All data loaded and augmented. Shape = ', np.shape(X))


  3%|▎         | 174/6612 [00:00<00:03, 1730.04it/s]

Axionable data Loaded. Shape =  (26449, 90, 250, 3)
Augmenting data... Wait...


100%|██████████| 6612/6612 [00:08<00:00, 743.99it/s]


Axionable data after augmentation. Shape =  (59509, 90, 250, 3)
Ironcar new track chicane Loaded. Shape =  (1519, 90, 250, 3)


 11%|█         | 170/1519 [00:00<00:00, 1692.58it/s]

Ironcar old track data Loaded. Shape =  (16028, 90, 250, 3)
Augmenting data... Wait...


100%|██████████| 4007/4007 [00:02<00:00, 1349.39it/s]


Ironcar data after augmentation. Shape =  (25561, 90, 250, 3)
All data loaded and augmented. Shape =  (85070, 90, 250, 3)


In [8]:
# Perform train/validation split. We do not use test set.
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=args['val_split'], random_state=12)

In [9]:
my_new_model.fit(
                X_train, 
                Y_train,
                nb_epoch=args['epochs'],
                batch_size=args['batch_size'], 
                verbose=1, 
                validation_data=(X_val, Y_val),
                shuffle=True)

W0528 15:31:47.976790 140715993876224 training.py:618] The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 68056 samples, validate on 17014 samples
Epoch 1/2
14000/68056 [=====>........................] - ETA: 16:22 - loss: 1.1586 - acc: 0.5439

KeyboardInterrupt: 

In [ ]:
import time
start=time.time()
for i in range(1,100):
    my_new_model.predict(X[i-1:i])

print(time.time()-start)